In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import warnings
warnings.filterwarnings('ignore')
from Preprocessing import date_splitter,time_trier,route,tod_departure

In [2]:
trainset = pd.read_csv(r'../Data/trainset.csv')
testset = pd.read_csv(r'../Data/testset.csv')

In [3]:
with open (r'../bin/features.pkl','rb') as f1:
    features=pickle.load(f1)

with open (r'../bin/encoder.pkl','rb') as f2:
    encoder=pickle.load(f2)   

# Outlier removal

In [4]:
#trainset['Std_price']=( trainset.Price - trainset.Price.mean())/trainset.Price.std()

In [5]:
#trainset = trainset[(trainset.Std_price<3) & (trainset.Std_price>-3)]
#trainset.drop(['Std_price'],axis=1, inplace=True)

In [6]:
y = pd.DataFrame(trainset.Price)
X = trainset.drop(['Price'], axis=1)
y_test = pd.DataFrame(testset.Price)
X_test = testset.drop(['Price'], axis=1)

In [7]:
features.fit(X)
X = pd.DataFrame(features.transform(X))
X_test = pd.DataFrame(features.transform(X_test))
encoder.fit(X)
X = pd.DataFrame(encoder.transform(X))
X_test = pd.DataFrame(encoder.transform(X_test))

In [8]:
for col in X.columns:
    X[col] = X[col].apply(lambda x:int(x))

In [9]:
for col in X.columns:
    X_test[col] = X_test[col].apply(lambda x:int(x))

In [10]:
import statsmodels.api as sm

X=sm.add_constant(X)

lr = sm.OLS(y.values,X).fit()
lr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.529
Model:                            OLS   Adj. R-squared:                  0.528
Method:                 Least Squares   F-statistic:                     399.1
Date:                Sun, 27 Jun 2021   Prob (F-statistic):               0.00
Time:                        17:14:08   Log-Likelihood:                -81013.
No. Observations:                8546   AIC:                         1.621e+05
Df Residuals:                    8521   BIC:                         1.623e+05
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -4.815e+04   4565.590    -10.546      0.000   -5.71e+04   -3.92e+04
0          -9605.7106    528.580    -18.173      0.000   -1.06e+04   -8569.567
1          -7522.7966    503.831    -14.931      0.000   -8510.428   -6535.165
2          -9344.4243    543.775    -17.184      0.000   -1.04e+04   -8278.494
3          -9403.6654    499.351    -18.832      0.000   -1.04e+04   -8424.817
4          -5108.4337    501.489    -10.187      0.000   -6091.473   -4125.395
5            4.13e+04   1400.002     29.499      0.000    3.86e+04     4.4e+04
6          -5086.0269    507.047    -10.031      0.000   -6079.962   -4092.092
7          -4982.3600    975.297     -5.109      0.000   -6894.178   -3070.542
8           -1.03e+04    506.526    -20.342      0.000   -1.13e+04   -9310.636
9          -1.234e+04   2957.985     -4.172      0.000   -1.81e+04   -6543.343
10         -8269.5757    517.264    -15.987      0.000   -9283.539   -7255.613
11         -7479.2956   1753.426     -4.266      0.000   -1.09e+04   -4042.156
12         -9867.2682    922.917    -10.691      0.000   -1.17e+04   -8058.128
13         -1.003e+04    923.846    -10.857      0.000   -1.18e+04   -8219.006
14         -9585.8290    917.475    -10.448      0.000   -1.14e+04   -7787.356
15         -9454.0284    934.226    -10.120      0.000   -1.13e+04   -7622.718
16         -9212.0589    916.912    -10.047      0.000    -1.1e+04   -7414.688
17          2230.1344    177.843     12.540      0.000    1881.520    2578.749
18            20.8837     17.435      1.198      0.231     -13.294      55.061
19          7.044e+04   5467.141     12.885      0.000    5.97e+04    8.12e+04
20         -2317.0685    178.488    -12.982      0.000   -2666.949   -1967.188
21          -181.6174     31.208     -5.819      0.000    -242.794    -120.441
22           171.3299      4.930     34.755      0.000     161.667     180.993
23            -4.3862      2.097     -2.091      0.037      -8.497      -0.275
24             4.0779      1.920      2.124      0.034       0.315       7.841
25             9.4015     18.867      0.498      0.618     -27.582      46.385
==============================================================================
Omnibus:                     4352.278   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            98883.463
Skew:                           1.943   Prob(JB):                         0.00
Kurtosis:                      19.205   Cond. No.                     1.04e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.48e-24. This might indicate that there are
strong multicollinearity problems or that the design matrix is 

In [11]:
X.drop(['const',18,25],axis=1, inplace=True)

In [12]:
from sklearn.linear_model import LinearRegression
lr= LinearRegression()
lr.fit(X, y)
X_test.drop([18,25],axis=1,inplace=True)
yp=lr.predict(X_test)


In [13]:
from sklearn.metrics import mean_absolute_percentage_error,r2_score
mean_absolute_percentage_error(testset.Price,yp)

0.2947417236033762

In [14]:
r2_score(testset.Price,yp)

0.5483921603367883

In [15]:
import xgboost as x
xx= x.XGBRegressor()
xx.fit(X,y)
yprex=xx.predict(X_test)
mean_absolute_percentage_error(y_test,yprex)

0.13896025986071822

In [16]:
from sklearn.ensemble import RandomForestRegressor 
rf= RandomForestRegressor()
rf.fit(X,y)
yprer=rf.predict(X_test)
mean_absolute_percentage_error(y_test,yprer)

0.13754750583745481

# You can't use MAE OR RMSE because the price of the flights change
# You have to use MAPE

In [17]:
X_test.dtypes

0     int64
1     int64
2     int64
3     int64
4     int64
5     int64
6     int64
7     int64
8     int64
9     int64
10    int64
11    int64
12    int64
13    int64
14    int64
15    int64
16    int64
17    int64
19    int64
20    int64
21    int64
22    int64
23    int64
24    int64
dtype: object